<a href="https://colab.research.google.com/github/MadhabBarman/Epidemic-Control-Model/blob/master/reprod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MadhabBarman/Epidemic-Control-Model.git

Cloning into 'Epidemic-Control-Model'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 58 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [2]:
cd Epidemic-Control-Model/

/content/Epidemic-Control-Model


In [6]:
import numpy as np
import numpy.linalg as la
from scipy.io import savemat, loadmat
import matplotlib.pyplot as plt
import pandas as pd
M = 16  #Total number of age-groups 
my_data = np.genfromtxt('data/age_structures/India-2019.csv', delimiter=',', skip_header=1)
aM, aF = my_data[:, 1], my_data[:, 2]
Ni=aM+aF;   Ni=Ni[0:M];  N=np.sum(Ni)
######################### contact matrices###################################
#Home contact Matrix
my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_home_1.xlsx', sheet_name='India',index_col=None)
CH = np.array(my_data)
#Workspace Contact Matrix
my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_work_1.xlsx', sheet_name='India',index_col=None)
CW = np.array(my_data)
#School Conact Matrix
my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_school_1.xlsx', sheet_name='India',index_col=None)
CS = np.array(my_data)
#Others Contact Matrix
my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_other_locations_1.xlsx', sheet_name='India',index_col=None)
CO = np.array(my_data)
CM = CH + CW + CS + CO

In [9]:
def r0_fun(beta, rho,xi, alpha_1, alpha_2, phi_qh, q, g_as, d_ar, phi_sh, d_sr, d_hr, eta, fsa, fsh):
    alpha_3 = 1-(alpha_1+alpha_2)
    F = np.array([[xi, 0, 0, 0, 0], [-alpha_1*xi, phi_qh, 0, 0, 0], [-alpha_2*xi, 0, g_as+d_ar, 0, 0],\
    [-alpha_3*xi,  0, -g_as, phi_sh+d_sr, 0], [0, -q*phi_qh, 0, -phi_sh, d_hr+eta]])
    Id = np.identity(M)
    Sigma = np.kron(F, Id)
    L0 = np.zeros((M, M))
    for i in range(M):
        for j in range(M):
            L0[i,j]=CM[i,j]*Ni[i]/Ni[j]
    T = np.zeros_like(Sigma)
    T[0:M, 2*M:3*M] = beta*L0
    T[0:M, 3*M:4*M] = fsa*beta*L0
    T[0:M, 4*M:5*M] = fsh*beta*(1-rho)*L0
    Sigma_inv = la.inv(Sigma)
    R = np.dot(T, Sigma_inv)
    #plt.spy(Sigma)
    return np.max(np.linalg.eigvals(R))

**Basic reproduction number $R_0$**

In [11]:
if __name__ == "__main__":
    alpha_d = 0.05    #fractional part from isolation
    beta  = 0.37      #rate if infection, feng
    rho = 0.75        #control parameter of H
    xi = 0.29         #recovery rate of E
    alpha_1 = 0.7    #fractional part of E-->Q
    alpha_2 = 0.2     #fractional part of E-->A
    alpha_3 = 1-(alpha_1+alpha_2)    #fractional part of E-->I
    phi_qh  = 1/10    #Recovery rate of Q
    q = 0.1           #fractional part of Q-->H
    g_as  = 0.1      #rate A-->I
    d_ar  = 2./7      #Recovery rate of A
    phi_sh = 1./2     #rate I-->H 
    d_sr =  1./7       #Recovery rate of I
    d_hr = (1-alpha_d)/10           #Recovery rate of H 
    eta = alpha_d/10  #Death rate
    fsa = 0.1        #Fraction of the contact matrix Cs 
    fsh = 0.1        #Fraction of the contact matrix Ch 
    u = r0_fun(beta, rho,xi, alpha_1, alpha_2, phi_qh, q, g_as, d_ar, phi_sh, d_sr, d_hr, eta, fsa, fsh)
    print(u)

(4.230353234752268+0j)
